# 🏀 QEPC NBA Sandbox (v1)

Interactive playground for:
- 📅 Loading schedule & picking games (next 3 days)
- 📊 Building team strengths (incl. injury overrides)
- λ Constructing λ tables (`lambda_df`)
- 🎯 Running single-script and multi-script QEPC simulations
- 🩺 Inspecting injury impacts and overrides

Use the sidebar outline to jump between sections.


## 🧩 1. Environment & Project Context


In [36]:
# 1. Environment & Project Context

from notebook_context import *  # sets project_root, adds qepc to sys.path, prints banner

import pandas as pd
import qepc_autoload as qa

print("Project root:", project_root)


Project root: C:\Users\wdors\qepc_project


---



## 📅 2. NBA Schedule & Game Selector (Next 3 Days)


In [ ]:
# 2. NBA Schedule & Game Selector (Next 3 Days)

import ipywidgets as widgets
from IPython.display import display

# Load full schedule
schedule = qa.load_nba_schedule()

# Filter to next 3 days
today = pd.Timestamp.today().normalize()
cutoff = today + pd.Timedelta(days=3)

games_window = schedule[
    (schedule["gameDate"] >= today) & (schedule["gameDate"] <= cutoff)
].copy()

print(f"Games between {today.date()} and {cutoff.date()}: {len(games_window)}")
display(games_window[["Date", "Away Team", "Home Team", "gameDate"]])

# Dropdown to pick a single game (optional)
if not games_window.empty:
    game_labels = [
        f"{row['Date']} – {row['Away Team']} @ {row['Home Team']}"
        for _, row in games_window.iterrows()
    ]
    dropdown = widgets.Dropdown(
        options=[("All games in window", "ALL")] +
                list(zip(game_labels, games_window.index)),
        description="Game:",
        layout=widgets.Layout(width="80%"),
    )
    display(dropdown)

    def select_games(selected):
        global games_to_model
        if selected == "ALL":
            games_to_model = games_window.copy()
        else:
            games_to_model = games_window.loc[[selected]].copy()
        print(f"\nGames to model: {len(games_to_model)}")
        display(games_to_model[["Date", "Away Team", "Home Team", "gameDate"]])

    def on_dropdown_change(change):
        if change["name"] == "value":
            select_games(change["new"])

    dropdown.observe(on_dropdown_change, names="value")

    # Initialize
    select_games("ALL")
else:
    print("No games found in the next 3 days.")
    games_to_model = pd.DataFrame()


---



## 📊 3. Team Strengths & Injury Overrides


In [ ]:
# 🧩 3. Team Strengths & Injury Overrides (recomputed here)

import pandas as pd
from qepc.sports.nba.strengths_v2 import calculate_advanced_strengths

# 3.1 Base advanced strengths
advanced_team_strengths = calculate_advanced_strengths()

print("Advanced team strengths (first 5 rows):")
display(advanced_team_strengths.head())

# 3.2 Load injury overrides (prefer data-driven, fallback to base)
inj_dd_path = project_root / "data" / "Injury_Overrides_data_driven.csv"
inj_base_path = project_root / "data" / "Injury_Overrides.csv"

injuries = None

if inj_dd_path.exists():
    print("\nUsing data-driven injury overrides:", inj_dd_path)
    injuries = pd.read_csv(inj_dd_path)
elif inj_base_path.exists():
    print("\nUsing base injury overrides:", inj_base_path)
    injuries = pd.read_csv(inj_base_path)
else:
    print("\n⚠️ No Injury_Overrides CSV found. Using raw strengths only.")

# 3.3 Build team_strengths_for_lambda
team_strengths_for_lambda = advanced_team_strengths.copy()

if injuries is not None:
    # Ensure required columns exist
    if "Team" not in injuries.columns:
        raise ValueError("Injury overrides file needs a 'Team' column.")
    if "Impact" not in injuries.columns:
        print("ℹ️ No 'Impact' column found in injuries; defaulting to 1.0.")
        injuries["Impact"] = 1.0

    # Compute a TEAM-LEVEL factor from player impacts
    # We treat Impact as an ORtg multiplier per injured player and multiply them,
    # with a floor so we don't totally nuke a team (e.g. min 0.60).
    def team_factor(series):
        prod = series.prod()
        return max(0.60, prod)  # floor at 0.60 so things don't go insane

    team_factors = (
        injuries.groupby("Team")["Impact"]
        .apply(team_factor)
        .reset_index()
        .rename(columns={"Impact": "ORtg_factor"})
    )

    print("\nTeam-level injury factors (first 10):")
    display(team_factors.head(10))

    # Merge onto strengths
    team_strengths_for_lambda = team_strengths_for_lambda.merge(
        team_factors, on="Team", how="left"
    )

    # Fill teams with no injuries → factor 1.0
    team_strengths_for_lambda["ORtg_factor"] = (
        team_strengths_for_lambda["ORtg_factor"].fillna(1.0)
    )

    # Keep raw ORtg for reference, apply factor to get adjusted ORtg
    team_strengths_for_lambda["ORtg_raw"] = team_strengths_for_lambda["ORtg"]
    team_strengths_for_lambda["ORtg"] = (
        team_strengths_for_lambda["ORtg_raw"] * team_strengths_for_lambda["ORtg_factor"]
    )

    print("\nteam_strengths_for_lambda (first 10 rows):")
    display(
        team_strengths_for_lambda[
            ["Team", "ORtg_raw", "ORtg", "ORtg_factor", "DRtg", "Pace", "Volatility"]
        ].head(10)
    )
else:
    print("\nteam_strengths_for_lambda = advanced_team_strengths (no injury adjustment).")
    display(team_strengths_for_lambda.head())


---



## 📐 4. Build λ Table (`lambda_df`) for Selected Games


In [ ]:
# 4. Build λ Table (lambda_df) for Selected Games

from qepc.core.lambda_engine import compute_lambda

if games_to_model.empty:
    print("No games_to_model defined or no games in window.")
else:
    lambda_df = compute_lambda(games_to_model, team_strengths_for_lambda)

    print("lambda_df preview:")
    display(
        lambda_df[
            ["Away Team", "Home Team", "lambda_away", "lambda_home", "vol_away", "vol_home"]
        ].head()
    )


---



## 🎛️ 5. Optional: Global λ Calibration (`lambda_calibrated`)


In [ ]:
# 5. Optional: Global λ Calibration (lambda_calibrated)

# Paste the scale_factor you computed from the backtest (example: 1.15)
cal = 1.15  # <-- replace with your scale_factor

try:
    lambda_df
except NameError:
    raise NameError("lambda_df is not defined. Run section 4 first.")

lambda_calibrated = lambda_df.copy()

for col in ["lambda_home", "lambda_away"]:
    if col in lambda_calibrated.columns:
        lambda_calibrated[col] = lambda_calibrated[col] * cal

print("Calibrated λ preview:")
display(
    lambda_calibrated[["Away Team", "Home Team", "lambda_away", "lambda_home"]].head()
)


---



## 🎯 6. Single-Script QEPC Simulation (Baseline)


In [ ]:
# 6. Single-Script QEPC Simulation (Baseline)

from qepc.core.simulator import run_qepc_simulation

# Use calibrated λ if available, else raw
lambda_input = lambda_calibrated if "lambda_calibrated" in globals() else lambda_df

sim_results = run_qepc_simulation(lambda_input, num_trials=20000)

print("Single-script QEPC sim results (first 5):")
display(
    sim_results[
        ["Away Team", "Home Team", "Home_Win_Prob", "Away_Win_Prob",
         "Expected_Score_Total", "Expected_Spread"]
    ].head()
)


---



## 🌌 7. Script Superposition Engine (Grind / Base / Chaos)


In [ ]:
# 7. Script Superposition Engine (Grind / Base / Chaos)

from qepc.core.simulator import run_qepc_simulation
import pandas as pd

SCRIPT_CONFIGS = [
    {
        "id": "GRIND",
        "name": "Grind (low total, low variance)",
        "lambda_scale": 0.92,
        "vol_scale": 0.90,
        "weight": 0.25,
    },
    {
        "id": "BASE",
        "name": "Base (normal game)",
        "lambda_scale": 1.00,
        "vol_scale": 1.00,
        "weight": 0.50,
    },
    {
        "id": "CHAOS",
        "name": "Chaos (high total, high variance)",
        "lambda_scale": 1.08,
        "vol_scale": 1.20,
        "weight": 0.25,
    },
]

total_w = sum(s["weight"] for s in SCRIPT_CONFIGS)
for s in SCRIPT_CONFIGS:
    s["weight"] = s["weight"] / total_w


def build_script_lambda(lambda_base: pd.DataFrame, script: dict) -> pd.DataFrame:
    df = lambda_base.copy()
    lam_scale = script["lambda_scale"]
    vol_scale = script["vol_scale"]

    for col in ["lambda_home", "lambda_away"]:
        if col in df.columns:
            df[col] = df[col] * lam_scale

    for col in ["vol_home", "vol_away"]:
        if col in df.columns:
            df[col] = df[col] * vol_scale

    return df


def run_qepc_multiscript(lambda_base: pd.DataFrame,
                         script_configs: list[dict],
                         num_trials: int = 20000) -> pd.DataFrame:
    script_results = []

    for script in script_configs:
        print(f"\nRunning script: {script['id']} – {script['name']}")
        lam_s = build_script_lambda(lambda_base, script)
        sim_s = run_qepc_simulation(lam_s, num_trials=num_trials)
        sim_s = sim_s.copy()
        sim_s["script_id"] = script["id"]
        sim_s["script_weight"] = script["weight"]
        script_results.append(sim_s)

    all_scripts_df = pd.concat(script_results, axis=0, ignore_index=True)

    candidate_keys = ["Date", "Time", "Away Team", "Home Team", "Venue", "Notes", "gameDate"]
    group_keys = [c for c in candidate_keys if c in all_scripts_df.columns]

    if not group_keys:
        all_scripts_df["game_idx"] = all_scripts_df.groupby("script_id").cumcount()
        group_keys = ["game_idx"]

    numeric_cols = all_scripts_df.select_dtypes(include="number").columns.tolist()
    numeric_cols = [c for c in numeric_cols if c not in ["script_weight"]]

    def weighted_agg(group: pd.DataFrame) -> pd.Series:
        weights = group["script_weight"]
        w = weights / weights.sum()

        result = {}
        for col in group.columns:
            if col in numeric_cols or col == "script_weight":
                continue
            if col in group_keys:
                continue
            result[col] = group[col].iloc[0]

        for col in numeric_cols:
            result[col] = (group[col] * w).sum()

        return pd.Series(result)

    combined = (
        all_scripts_df
        .groupby(group_keys, as_index=False)
        .apply(weighted_agg)
        .reset_index(drop=True)
    )

    print("\n✅ Multi-script QEPC results computed.")
    return combined


---



## ⚖️ 8. Multi-Script QEPC Simulation & Comparison


In [ ]:
# 8. Multi-Script QEPC Simulation & Comparison

lambda_input = lambda_calibrated if "lambda_calibrated" in globals() else lambda_df

multi_script_results = run_qepc_multiscript(
    lambda_base=lambda_input,
    script_configs=SCRIPT_CONFIGS,
    num_trials=20000,
)

print("Multi-script results (first 5):")
display(
    multi_script_results[
        ["Away Team", "Home Team", "Home_Win_Prob", "Away_Win_Prob",
         "Expected_Score_Total", "Expected_Spread"]
    ].head()
)

# Optional: compare single-script vs multi-script
import pandas as pd

single = sim_results.copy()
multi = multi_script_results.copy()

key_cols = ["Away Team", "Home Team"]
metric_cols = ["Home_Win_Prob", "Away_Win_Prob", "Expected_Score_Total", "Expected_Spread"]

single_ren = single[key_cols + metric_cols].rename(
    columns={c: f"{c}_single" for c in metric_cols}
)
multi_ren = multi[key_cols + metric_cols].rename(
    columns={c: f"{c}_multi" for c in metric_cols}
)

compare_df = pd.merge(single_ren, multi_ren, on=key_cols, how="inner")

for c in metric_cols:
    compare_df[f"{c}_delta"] = compare_df[f"{c}_multi"] - compare_df[f"{c}_single"]

print("\nSingle vs Multi-script comparison:")
display(compare_df)


---



## 🩺 9. Injury Impact Inspector


In [ ]:
# 9. Injury Impact Inspector

dd_inj_path = project_root / "data" / "Injury_Overrides_data_driven.csv"

if dd_inj_path.exists():
    inj_dd = pd.read_csv(dd_inj_path)
    print("Data-driven injury overrides loaded from:", dd_inj_path)
    display(inj_dd.head())

    def show_injury_impact(team: str, player: str):
        row = inj_dd[(inj_dd["Team"] == team) & (inj_dd["PlayerName"] == player)]
        if row.empty:
            print(f"No injury override found for {player} on {team}")
        else:
            display(row)

    # Example calls (edit as needed)
    show_injury_impact("Indiana Pacers", "Tyrese Haliburton")
    show_injury_impact("Boston Celtics", "Jayson Tatum")
else:
    print("No Injury_Overrides_data_driven.csv found; run the injury override notebook first.")


---

